In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

In [3]:
#Read in the original csv file
PittCits = pd.read_csv('../data/PittsburgNonTrafficCitations.csv', low_memory=False)

In [8]:
PittCits.head()

,PK,CCR,GENDER,RACE,AGE,CITEDTIME,INCIDENTLOCATION,OFFENSES,NEIGHBORHOOD,ZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
2964,1083085,17153868,M,B,19.0,2014-08-14 17:05:00,"FRANKSTOWN AV & COLLIER ST PITTSBURGH, PA",601.12 Interference with Official Duties,Homewood North,5,1302,9.0,2.0,-79.891902,40.456393
1555,1080475,17041523,M,W,51.0,2015-03-31 12:00:00,"200 Block 5TH AV PITTSBURGH, PA",2709(a)(3) Harassment No Legitimate Purpose,Central Business District,2,201,6.0,6.0,-80.002074,40.441228
41,1077920,16000875,M,B,28.0,2016-01-02 12:15:00,"800 Block MT PLEASANT RD PITTSBURGH, PA",5503(a)(1) DISORDERLY CONDUCT - ENGAGE IN FIGH...,Northview Heights,1,2609,1.0,1.0,-79.999473,40.475744
1088,1079659,17008684,F,B,52.0,2016-01-14 22:00:00,"BROWNSVILLE RD & BAUSMAN ST PITTSBURGH, PA","2709(a)(1) Harassment by Physical Contact, or ...",Knoxville,3,3001,NaN,NaN,-79.988039,40.414434
39,1077918,16002105,F,B,52.0,2016-01-15 21:30:00,"10 Block CARRICK AV PITTSBURGH, PA",601.16(b)(1)(a) Any Person who urinates or def...,Carrick,3,2901,4.0,3.0,-79.982973,40.387237


In [23]:
#convert times to datetimes and sort
PittCits['CITEDTIME'] = pd.to_datetime(PittCits['CITEDTIME'])
PittCits = PittCits.sort_values(['CITEDTIME'], ascending=True)
PittCits.head(50)

,PK,CCR,GENDER,RACE,AGE,CITEDTIME,INCIDENTLOCATION,OFFENSES,NEIGHBORHOOD,ZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
2964,1083085,17153868,M,B,19.0,2014-08-14 17:05:00,"FRANKSTOWN AV & COLLIER ST PITTSBURGH, PA",601.12 Interference with Official Duties,Homewood North,5,1302,9.0,2.0,-79.891902,40.456393
1555,1080475,17041523,M,W,51.0,2015-03-31 12:00:00,"200 Block 5TH AV PITTSBURGH, PA",2709(a)(3) Harassment No Legitimate Purpose,Central Business District,2,201,6.0,6.0,-80.002074,40.441228
41,1077920,16000875,M,B,28.0,2016-01-02 12:15:00,"800 Block MT PLEASANT RD PITTSBURGH, PA",5503(a)(1) DISORDERLY CONDUCT - ENGAGE IN FIGH...,Northview Heights,1,2609,1.0,1.0,-79.999473,40.475744
1088,1079659,17008684,F,B,52.0,2016-01-14 22:00:00,"BROWNSVILLE RD & BAUSMAN ST PITTSBURGH, PA","2709(a)(1) Harassment by Physical Contact, or ...",Knoxville,3,3001,NaN,NaN,-79.988039,40.414434
39,1077918,16002105,F,B,52.0,2016-01-15 21:30:00,"10 Block CARRICK AV PITTSBURGH, PA",601.16(b)(1)(a) Any Person who urinates or def...,Carrick,3,2901,4.0,3.0,-79.982973,40.387237
38,1077917,16002105,F,B,52.0,2016-01-15 21:30:00,"2300 Block BROWNSVILLE RD PITTSBURGH, PA",5505 Public Drunkenness,Carrick,3,2901,4.0,3.0,-79.985520,40.389049
40,1077919,16012698,M,W,57.0,2016-01-21 18:30:00,"800 Block LIBERTY AV PITTSBURGH, PA",5505 Public Drunkenness,Golden Triangle/Civic Arena,2,201,6.0,6.0,-79.998927,40.442627
78,1077888,16017044,M,W,21.0,2016-01-29 02:00:00,"200 Block ATWOOD ST PITTSBURGH, PA","6501(a)(1) Scattering Rubbish; Any waste, dang...",Central Oakland,4,405,3.0,3.0,-79.956889,40.440420
245,1077881,16037288,M,B,51.0,2016-02-03 12:00:00,"100 Block GREENFIELD AV PITTSBURGH, PA",2709(a)(3) Harassment No Legitimate Purpose,Greenfield,4,1516,5.0,3.0,-79.951937,40.424897
45,1077924,16029759,M,W,19.0,2016-02-10 16:00:00,"1300 Block BREINING ST PITTSBURGH, PA",633.05 Rabies Vaccination,Brookline,6,1918,4.0,5.0,-80.010699,40.391198


In [ ]:
#Periodical Cicadas are confirmed in Pittsburgh for Brood V (2016), but there are nearby confirmations 
#     for Brood VIII (2019).  Including data around both years in this dataframe.

In [13]:
PittCitstrim = PittCits[PittCits['CITEDTIME'] > dt.datetime(2015,5,1)]
PittCitstrim = PittCitstrim[PittCitstrim['CITEDTIME'] < dt.datetime(2020,7,1)]
PittCitstrim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10411 entries, 41 to 10810
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     10411 non-null  int64         
 1   CCR                    10411 non-null  object        
 2   GENDER                 10375 non-null  object        
 3   RACE                   10374 non-null  object        
 4   AGE                    10373 non-null  float64       
 5   CITEDTIME              10411 non-null  datetime64[ns]
 6   INCIDENTLOCATION       10411 non-null  object        
 7   OFFENSES               10411 non-null  object        
 8   NEIGHBORHOOD           10276 non-null  object        
 9   ZONE                   10411 non-null  object        
 10  INCIDENTTRACT          10280 non-null  object        
 11  COUNCIL_DISTRICT       9422 non-null   float64       
 12  PUBLIC_WORKS_DIVISION  9422 non-null   float64       
 13  

In [12]:
#2015
#0 entries??? Reviewing the original file suggest to me that they didn't really 
#   start tracking citations here until 2016
PittCits2015 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2015,5,1)]
PittCits2015 = PittCits2015[PittCits2015['CITEDTIME'] < dt.datetime(2015,7,1)]
PittCits2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     0 non-null      int64         
 1   CCR                    0 non-null      object        
 2   GENDER                 0 non-null      object        
 3   RACE                   0 non-null      object        
 4   AGE                    0 non-null      float64       
 5   CITEDTIME              0 non-null      datetime64[ns]
 6   INCIDENTLOCATION       0 non-null      object        
 7   OFFENSES               0 non-null      object        
 8   NEIGHBORHOOD           0 non-null      object        
 9   ZONE                   0 non-null      object        
 10  INCIDENTTRACT          0 non-null      object        
 11  COUNCIL_DISTRICT       0 non-null      float64       
 12  PUBLIC_WORKS_DIVISION  0 non-null      float64       
 13  X                

In [15]:
#2016
#23 entries
PittCits2016 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2016,5,1)]
PittCits2016 = PittCits2016[PittCits2016['CITEDTIME'] < dt.datetime(2016,7,1)]
PittCits2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 13 to 176
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     23 non-null     int64         
 1   CCR                    23 non-null     object        
 2   GENDER                 23 non-null     object        
 3   RACE                   23 non-null     object        
 4   AGE                    23 non-null     float64       
 5   CITEDTIME              23 non-null     datetime64[ns]
 6   INCIDENTLOCATION       23 non-null     object        
 7   OFFENSES               23 non-null     object        
 8   NEIGHBORHOOD           21 non-null     object        
 9   ZONE                   23 non-null     object        
 10  INCIDENTTRACT          21 non-null     object        
 11  COUNCIL_DISTRICT       22 non-null     float64       
 12  PUBLIC_WORKS_DIVISION  22 non-null     float64       
 13  X    

In [17]:
#2017
#557 entries
PittCits2017 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2017,5,1)]
PittCits2017 = PittCits2017[PittCits2017['CITEDTIME'] < dt.datetime(2017,7,1)]
PittCits2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 557 entries, 2192 to 2730
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     557 non-null    int64         
 1   CCR                    557 non-null    object        
 2   GENDER                 553 non-null    object        
 3   RACE                   555 non-null    object        
 4   AGE                    554 non-null    float64       
 5   CITEDTIME              557 non-null    datetime64[ns]
 6   INCIDENTLOCATION       557 non-null    object        
 7   OFFENSES               557 non-null    object        
 8   NEIGHBORHOOD           547 non-null    object        
 9   ZONE                   557 non-null    object        
 10  INCIDENTTRACT          547 non-null    object        
 11  COUNCIL_DISTRICT       530 non-null    float64       
 12  PUBLIC_WORKS_DIVISION  530 non-null    float64       
 13  X

In [18]:
#2018
#437 entries
PittCits2018 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2018,5,1)]
PittCits2018 = PittCits2018[PittCits2018['CITEDTIME'] < dt.datetime(2018,7,1)]
PittCits2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 437 entries, 5396 to 5417
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     437 non-null    int64         
 1   CCR                    437 non-null    object        
 2   GENDER                 435 non-null    object        
 3   RACE                   436 non-null    object        
 4   AGE                    434 non-null    float64       
 5   CITEDTIME              437 non-null    datetime64[ns]
 6   INCIDENTLOCATION       437 non-null    object        
 7   OFFENSES               437 non-null    object        
 8   NEIGHBORHOOD           426 non-null    object        
 9   ZONE                   437 non-null    object        
 10  INCIDENTTRACT          426 non-null    object        
 11  COUNCIL_DISTRICT       385 non-null    float64       
 12  PUBLIC_WORKS_DIVISION  385 non-null    float64       
 13  X

In [21]:
#2019
#485 entries
PittCits2019 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2019,5,1)]
PittCits2019 = PittCits2019[PittCits2019['CITEDTIME'] < dt.datetime(2019,7,1)]
PittCits2019.head(30)

,PK,CCR,GENDER,RACE,AGE,CITEDTIME,INCIDENTLOCATION,OFFENSES,NEIGHBORHOOD,ZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
7976,1092541,19081552,F,B,38.0,2019-05-01 08:00:00,"7200 Block FORMOSA WY PITTSBURGH, PA","2709(a)(1) Harassment by Physical Contact, or ...",Homewood South,5,1303,9.0,2.0,-79.897209,40.455061
7588,1091801,19081671,M,W,58.0,2019-05-01 13:28:00,"5TH AV & THOMAS BL PITTSBURGH, PA",601.16(b)(1)(a) Any Person who urinates or def...,Larimer,5,1208,9.0,2.0,-79.911422,40.454923
7589,1091802,19081897,M,B,49.0,2019-05-01 17:49:00,"7200 Block FRANKSTOWN AV PITTSBURGH, PA",601.08(a) Alcohol or Liquor Consumption on Str...,Homewood South,5,1303,9.0,2.0,-79.896270,40.457389
7590,1091803,19082033,M,B,33.0,2019-05-01 20:05:00,"E CARSON ST & S 23RD ST PITTSBURGH, PA",5505 Public Drunkenness,South Side Flats,3,1609,3.0,3.0,-79.973040,40.428328
7591,1091804,19082277,M,B,51.0,2019-05-02 03:13:00,"W CARSON ST & CORLISS STREET TUN PITTSBURGH, PA","5507(a) Obstructing highway, railroad or publi...",Sheraden,6,2018,NaN,NaN,0.000000,0.000000
7592,1091805,19082647,M,B,15.0,2019-05-02 16:00:00,"REED ST & BRACKENRIDGE ST PITTSBURGH, PA",5503(a)(1) DISORDERLY CONDUCT - ENGAGE IN FIGH...,Middle Hill,2,501,6.0,3.0,-79.971867,40.445001
7593,1091806,19082821,M,B,21.0,2019-05-02 20:20:00,"60 Block BRACEY DR PITTSBURGH, PA",627.02(b)(i) Possessing a small amount of mari...,East Hills,5,1306,NaN,NaN,0.000000,0.000000
7594,1091807,19082821,M,B,32.0,2019-05-02 20:20:00,"60 Block BRACEY DR PITTSBURGH, PA",627.02(b)(i) Possessing a small amount of mari...,East Hills,5,1306,NaN,NaN,0.000000,0.000000
7620,1091833,19082877,M,B,59.0,2019-05-02 21:20:00,"1100 Block BROWNSVILLE RD PITTSBURGH, PA",5505 Public Drunkenness,Carrick,3,2902,4.0,3.0,-79.990903,40.404947
7596,1091809,19083036,M,W,26.0,2019-05-03 02:10:00,"SIDNEY ST & S 18TH ST PITTSBURGH, PA",5505 Public Drunkenness,South Side Flats,3,1702,3.0,3.0,-79.980472,40.429687


In [20]:
#2020
#138 entries
PittCits2020 = PittCitstrim[PittCitstrim['CITEDTIME'] > dt.datetime(2020,5,1)]
PittCits2020 = PittCits2020[PittCits2020['CITEDTIME'] < dt.datetime(2020,7,1)]
PittCits2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 10255 to 10810
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PK                     138 non-null    int64         
 1   CCR                    138 non-null    object        
 2   GENDER                 137 non-null    object        
 3   RACE                   137 non-null    object        
 4   AGE                    137 non-null    float64       
 5   CITEDTIME              138 non-null    datetime64[ns]
 6   INCIDENTLOCATION       138 non-null    object        
 7   OFFENSES               138 non-null    object        
 8   NEIGHBORHOOD           137 non-null    object        
 9   ZONE                   138 non-null    object        
 10  INCIDENTTRACT          137 non-null    object        
 11  COUNCIL_DISTRICT       125 non-null    float64       
 12  PUBLIC_WORKS_DIVISION  125 non-null    float64       
 13 